In [ ]:
import os
import datetime
import firebirdsql
from queue import Queue
import concurrent.futures
from openpyxl import Workbook
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

# ===================== Funções de Conexão =====================
def get_firebird_connection():
    # Ajuste os parâmetros conforme sua configuração, inclusive charset
    return firebirdsql.connect(
        host=os.getenv('HOST'),
        port=int(os.getenv('PORT', '3050')),
        database=os.getenv('DB_PATH'),
        user=os.getenv('APP_USER'),
        password=os.getenv('PASSWORD'),
        role=os.getenv('ROLE'),
        auth_plugin_name=os.getenv('AUTH'),
        wire_crypt=False,
        charset='ISO8859_1'
    )

def create_connection_pool(pool_size=20):
    """
    Cria um pool de conexões Firebird com pool_size fixo (20).
    """
    pool = Queue(maxsize=pool_size)
    for _ in range(pool_size):
        conn = get_firebird_connection()
        pool.put(conn)
    return pool

def get_connection_from_pool(pool):
    # Fica bloqueado até ter uma conexão disponível no pool
    return pool.get()

def release_connection_to_pool(pool, conn):
    # Devolve a conexão ao pool
    pool.put(conn)

def get_total_count(pool):
    """
    Retorna o número total de registros da tabela PRODUTO.
    """
    conn = get_connection_from_pool(pool)
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT COUNT(*) FROM PRODUTO")
        count = cursor.fetchone()[0]
        return count
    finally:
        release_connection_to_pool(pool, conn)

# ===================== Função para buscar um "chunk" de registros =====================
def fetch_chunk(offset, chunk_size, pool):
    """
    Busca um pedaço (chunk) de registros da tabela PRODUTO.
    Usa a sintaxe Firebird: FIRST {chunk_size} SKIP {offset}
    """
    conn = get_connection_from_pool(pool)
    try:
        cursor = conn.cursor()
        query = (
            "SELECT CDPRODUTO, NUMORIGINAL, DESCRICAO, PRECOCUSTO, PRECOVENDA, "
            "ESTOQUEPREVISTO, UNIDADE, NCM, LOCALIZACAO "
            "FROM PRODUTO FIRST {} SKIP {}"
        ).format(chunk_size, offset)
        cursor.execute(query)
        rows = cursor.fetchall()
        return rows
    finally:
        release_connection_to_pool(pool, conn)

# ===================== Mapeamento de cada registro para a linha do Excel =====================
def process_record(db_row, current_date_str):
    """
    Recebe uma tupla (db_row) com os campos extraídos da tabela PRODUTO:
      db_row = (CDPRODUTO, NUMORIGINAL, DESCRICAO, PRECOCUSTO, PRECOVENDA, 
                ESTOQUEPREVISTO, UNIDADE, NCM, LOCALIZACAO)
    Retorna uma lista com 38 itens conforme o mapeamento:
      0. CODIGO SEQUENCIAL (vazio)
      1. Nome da Empresa -> "comagro"
      2. CODIGO SEQUENCIAL DO ITEM -> CDPRODUTO
      3. NOME GRUPO -> "TEMPORARIO"
      4. NOME DESCRICAO -> "TEMPORARIO"
      5. NOME FABRICANTE -> "DIVERSOS"
      6. Nº FABRICANTE -> NUMORIGINAL
      7. CÓDIGO DE BARRAS (EAN 13) -> "SEM GTIN"
      8. CÓDIGO DE BARRAS (qualquer) -> "SEM GTIN"
      9. APLICAÇÃO DO PRODUTO -> DESCRICAO
     10. INFORMAÇÕES ADICIONAIS (APLICAÇÃO 2) -> "SIMILARES:" (a ser ajustado)
     11. Alíquota de IPI -> 0
     12. Situação tributária do IPI -> "060"
     13. Alíquota de ICMS na Entrada -> vazio
     14. Peso da peça -> vazio
     15. Unidade da peça -> UNIDADE
     16. Código ANP -> vazio
     17. Quantidade Embalagem de Compra -> 1
     18. Quantidade Embalagem de Venda -> 1
     19. Classificação Fiscal -> NCM
     20. Preço de tabela -> vazio
     21. Data do Preço de tabela -> vazio
     22. Data do Cadastro -> data atual (current_date_str)
     23. Origem do Produto -> 0
     24. Situação Tributária do Item -> "00"
     25. Percentual da Alíquota ICMS -> "20,5%"
     26. Preço Custo -> PRECOCUSTO
     27. Preço venda -> PRECOVENDA
     28. Quantidade Estoque -> ESTOQUEPREVISTO
     29. Posição fixa do item -> LOCALIZACAO
     30 a 37. Restantes (Margem, Mkup, Estoque crítico, etc.) -> vazios
    """
    return [
        "",                   # 0. CODIGO SEQUENCIAL (não precisa digitar)
        "comagro",            # 1. Nome da Empresa
        db_row[0],            # 2. CODIGO SEQUENCIAL DO ITEM (CDPRODUTO)
        "TEMPORARIO",         # 3. NOME GRUPO
        "TEMPORARIO",         # 4. NOME DESCRICAO
        "DIVERSOS",           # 5. NOME FABRICANTE
        db_row[1],            # 6. Nº FABRICANTE (NUMORIGINAL)
        "SEM GTIN",           # 7. CÓDIGO DE BARRAS (EAN 13)
        "SEM GTIN",           # 8. CÓDIGO DE BARRAS (qualquer)
        db_row[2],            # 9. APLICAÇÃO DO PRODUTO (DESCRICAO)
        "SIMILARES:",         # 10. INFORMAÇÕES ADICIONAIS (ficará fixo por enquanto)
        0,                    # 11. Alíquota de IPI
        "060",                # 12. Situação tributária do IPI
        "",                   # 13. Alíquota de ICMS na Entrada (vazio)
        "",                   # 14. Peso da peça (vazio)
        db_row[6],            # 15. Unidade da peça (UNIDADE)
        "",                   # 16. Código ANP (vazio)
        1,                    # 17. Quantidade Embalagem de Compra
        1,                    # 18. Quantidade Embalagem de Venda
        db_row[7],            # 19. Classificação Fiscal (NCM)
        "",                   # 20. Preço de tabela (vazio)
        "",                   # 21. Data do Preço de tabela (vazio)
        current_date_str,     # 22. Data do Cadastro
        0,                    # 23. Origem do Produto
        "00",                 # 24. Situação Tributária do Item
        "20,5%",              # 25. Percentual da Alíquota ICMS
        db_row[3],            # 26. Preço Custo (PRECOCUSTO)
        db_row[4],            # 27. Preço venda (PRECOVENDA)
        db_row[5],            # 28. Quantidade Estoque (ESTOQUEPREVISTO)
        db_row[8],            # 29. Posição fixa (LOCALIZACAO)
        "",                   # 30. Margem de Lucro do item (vazio)
        "",                   # 31. Mkup para preço atacado (vazio)
        "",                   # 32. Mkup para preço varejo (vazio)
        "",                   # 33. Mkup Mínimo (vazio)
        "",                   # 34. Estoque crítico (vazio)
        "",                   # 35. Estoque máximo (vazio)
        "",                   # 36. Quantidade de dias (máximo) (vazio)
        ""                    # 37. Quantidade de dias (mínimo) (vazio)
    ]

# ===================== Função Principal =====================
def main():
    # Cria pool de conexões
    pool = create_connection_pool(pool_size=20)
    
    # Obtém a quantidade total de registros da tabela PRODUTO
    total_count = get_total_count(pool)
    print(f"Total de registros a extrair: {total_count}")
    
    # Define o tamanho do chunk (pode ser ajustado conforme volume de dados)
    chunk_size = 100
    offsets = range(0, total_count, chunk_size)
    
    # Data atual formatada (usada para "Data do Cadastro")
    current_date_str = datetime.datetime.now().strftime("%d/%m/%Y")
    
    all_data_rows = []  # aqui serão armazenadas todas as linhas processadas
    
    # Usa ThreadPoolExecutor para processar os chunks em paralelo
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        # Mapeia cada tarefa (chunk) com seu offset
        future_to_offset = {
            executor.submit(fetch_chunk, offset, chunk_size, pool): offset
            for offset in offsets
        }
        for future in concurrent.futures.as_completed(future_to_offset):
            offset = future_to_offset[future]
            try:
                chunk = future.result()
                print(f"Processado chunk com offset {offset} (tamanho: {len(chunk)})")
                # Para cada registro, aplica o mapeamento para a linha do Excel
                for db_row in chunk:
                    excel_row = process_record(db_row, current_date_str)
                    all_data_rows.append(excel_row)
            except Exception as exc:
                print(f"Erro ao processar chunk com offset {offset}: {exc}")
    
    print(f"Total de registros processados: {len(all_data_rows)}")
    
    # ===================== Criação do Excel =====================
    wb = Workbook()
    ws = wb.active
    
    # Primeira linha de cabeçalho (38 colunas conforme especificado)
    header1 = [
        "CODIGO SEQUENCIAL Não precisa digitar",
        "Nome da Empresa - Máximo 50 caracteres",
        "CODIGO SEQUENCIAL DO ITEM UTILIZADO ATUALMENTE",
        "NOME GRUPO Máximo 50 Caracteres",
        "NOME DESCRICAO Máximo 50 Caracteres",
        "NOME FABRICANTE Máximo 50 Caracteres",
        "NÚMERO DO FABRICANTE - MÁXIMO 20 CARACTERES",
        "CÓDIGO DE BARRAS - Máximo 13 caracteres - Formato EAN 13",
        "CÓDIGO DE BARRAS - Máximo 13 caracteres - Qualquer formato",
        "APLICAÇÃO DO PRODUTO - Máximo 100 CARACTERES",
        "INFORMAÇÕES ADICIONAIS DO PRODUTO (APLICAÇÃO 2) - Máximo 250 CARACTERES",
        "Alíquota de IPI se o produto possuir - Formato Percentual",
        "Situação tributária do IPI - Máximo 2 caracteres",
        "Alíquota de ICMS na Entrada se o produto possuir - Formato Percentual",
        "Peso da peça - Formato decimal (duas casas após a vírgula)",
        "Unidade da peça - MÁXIMO 2 CARACTERES",
        "Código ANP para Combustíveis ou Derivados de petróleo - Formato inteiro 9 caracteres",
        "Quantidade Embalagem de Compra - Formato Decimal",
        "Quantidade Embalagem de Venda - Formato Decimal",
        "Classificação Fiscal do produto - MÁXIMO 15 CARACTERES",
        "Preço de tabela - Formato Decimal (duas casas após a vírgula)",
        "Data do Preço de tabela - Campo Data DD/MM/AAAA",
        "Data do Cadastro Campo Data DD/MM/AAAA",
        "Origem do Produto - Máximo 1 caracter",
        "Situação Tributária do Item",
        "Percentual da Alíquota ICMS",
        "Preço Custo do item - Formato Decimal (duas casas após a vírgula)",
        "Preço venda do item - Formato Decimal (duas casas após a vírgula)",
        "Quantidade Estoque - Formato Decimal",
        "Posição fixa do item - Máximo 30 caracteres",
        "Margem de Lucro do item - Formato Percentual",
        "Mkup do Item para formação de preço atacado - Formato Percentual",
        "Mkup do Item para formação de preço varejo - Formato Percentual",
        "Mkup Mínimo do item",
        "Estoque crítico da peça",
        "Estoque máximo da peça",
        "Quantidade de dias (máximo)",
        "Quantidade de dias (mínimo)"
    ]
    ws.append(header1)
    
    # Segunda linha de cabeçalho (com cuidado com as colunas vazias)
    # Conforme informado: a linha possui apenas alguns títulos nos índices determinados.
    second_header = ["" for _ in range(38)]
    second_header[1] = "CÓD INT(SIST ANTIGO)"
    second_header[2] = "GRUPOS"
    second_header[3] = "DESCRIÇÃO(SUBGRUPO)"
    second_header[4] = "FABRICANTE(MARCA)"
    second_header[5] = "Nº FABRICANTE"
    second_header[7] = "APLICAÇÃO"
    second_header[37] = "NCM"
    ws.append(second_header)
    
    # Insere as linhas de dados (cada linha já com 38 colunas)
    for row in all_data_rows:
        ws.append(row)
    
    # Salva o arquivo Excel
    output_filename = "produtos.xlsx"
    wb.save(output_filename)
    print(f"Arquivo Excel salvo como '{output_filename}'")
    
    # Fecha todas as conexões do pool
    while not pool.empty():
        conn = pool.get()
        conn.close()

if __name__ == "__main__":
    main()